In [28]:
import json
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np

import re, string, unicodedata
import nltk
import contractions
import inflect
import sklearn
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [29]:


comment_dict = {}

# filtering data
for line in open('RC_2015-01/test_data.txt'):
    line_short = line[:-2]
    json_file = json.loads(line_short)
    comment_id =  json_file['id']
    comment_body = json_file['body']
    comment_sub = json_file['subreddit']
    comment = {"body": comment_body, "subreddit": comment_sub}
    comment_dict[comment_id] = comment


#writing filtered data to file
with open('result.json', 'w') as fp:
    json.dump(comment_dict, fp)
    
#sanity check
with open('result.json', 'r') as rf:
    data = json.load(rf)
    for entry in data:
        print(data[entry])
    

{'body': 'A quarry', 'subreddit': 'sandiego'}
{'body': "[Salutations! I'm not sure what you said.](http://imgur.com/9TtaInH) \n \n", 'subreddit': 'RWBY'}
{'body': 'I got into baseball at about he same time Matt Cain started playing in the majors. Crazy to see him go. I teared up a bit too.', 'subreddit': 'baseball'}
{'body': 'FUCKING TORY', 'subreddit': '2007scape'}
{'body': 'I see a water dragon ', 'subreddit': 'mildlyinteresting'}
{'body': 'Wait. The Michigan what? Where is this? Is this like U of M club or a just state of Michigan?', 'subreddit': 'Cubers'}
{'body': 'ye fam', 'subreddit': 'teenagers'}
{'body': '143417804| &gt; United States Anonymous (ID: LIAKFEVH)\n\n&gt;&gt;143412250 (OP)\noldfag here\n\n2016: Hillary\n2012: Obama\n2008: Obama\n2004: Kerry\n2000: Buchanan\n1996: Dole\n1992: Bush\n1988: Bush\n1984: Reagan\n\t\t\t', 'subreddit': '4chan4trump'}
{'body': 'That is some chicken salad outta chicken shit running. ', 'subreddit': 'CFB'}
{'body': 'Does he even know the rules

{'body': "Okay thanks. I'm in phx so won't get my hopes up but hopefully will be pleasantly surprised! \n\nHas anyone figured out if these are just repackaged Gardein or other brand? They've been out of stock for sooo long I can't recall. ", 'subreddit': 'vegan'}
{'body': '[+BriannaMGoux](https://www.reddit.com/r/IAmA/comments/737yli/i_am_brianna_goux_a_19_year_old_redditor_who_has/dnop508/):\n\nRight? lol', 'subreddit': 'ConciseIAmA'}
{'body': '[deleted]', 'subreddit': 'kotor'}
{'body': '907', 'subreddit': 'AskReddit'}
{'body': 'Even better, involve her, let HER send out the invitations to those people. Make her think its about making amends to HER, that it\'s about HER. \n\nThat way when DH does the mic drop moment with a speech ending "And of course I\'d like to thank my mother for making all of this possible, after all if it wasn\'t for her lies about us having a shotgun wedding and a miscarriage none of us would be here tonight" it hit\'s her fully. ', 'subreddit': 'JUSTNOMIL'}
{'

In [31]:
def remove_non_ascii(words):
    #"""Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    #"""Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    #"""Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    #"""Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    #"""Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    #"""Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    #"""Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas


In [33]:


with open('result.json', 'r') as rf:
    data = json.load(rf)
    for entry in data:
        doc_sample = data[entry]['body']
        print('original document: ')
        words = []
        for word in doc_sample.split(' '):
            words.append(word)
        print(words)
        
        words = normalize(words)
        print('\n\n tokenized and lemmatized document: ')
        print(stem_and_lemmatize(words))

original document: 
['A', 'quarry']


 tokenized and lemmatized document: 
(['quarry'], ['quarry'])
original document: 
['[Salutations!', "I'm", 'not', 'sure', 'what', 'you', 'said.](http://imgur.com/9TtaInH)', '\n', '\n']


 tokenized and lemmatized document: 
(['salut', 'im', 'sur', 'saidhttpimgurcom9ttainh', '\n', '\n'], ['salutations', 'im', 'sure', 'saidhttpimgurcom9ttainh', '\n', '\n'])
original document: 
['I', 'got', 'into', 'baseball', 'at', 'about', 'he', 'same', 'time', 'Matt', 'Cain', 'started', 'playing', 'in', 'the', 'majors.', 'Crazy', 'to', 'see', 'him', 'go.', 'I', 'teared', 'up', 'a', 'bit', 'too.']


 tokenized and lemmatized document: 
(['got', 'basebal', 'tim', 'mat', 'cain', 'start', 'play', 'maj', 'crazy', 'see', 'go', 'tear', 'bit'], ['get', 'baseball', 'time', 'matt', 'cain', 'start', 'play', 'major', 'crazy', 'see', 'go', 'tear', 'bite'])
original document: 
['FUCKING', 'TORY']


 tokenized and lemmatized document: 
(['fuck', 'tory'], ['fuck', 'tory'])
origina



 tokenized and lemmatized document: 
(['remov'], ['remove'])
original document: 
['OP,', 'what', 'did', 'you', 'do?']


 tokenized and lemmatized document: 
(['op'], ['op'])
original document: 
['Yes,', 'also', 'if', 'you', 'get', 'P', 'grade', 'in', 'all', 'levels', '(not', 'bosses)', 'you', 'unlock', 'a', 'filter']


 tokenized and lemmatized document: 
(['ye', 'also', 'get', 'p', 'grad', 'level', 'boss', 'unlock', 'filt'], ['yes', 'also', 'get', 'p', 'grade', 'level', 'boss', 'unlock', 'filter'])
original document: 
['That', 'part', "couldn't", 'be', 'anymore', 'irrelevant.', "I'm", 'talking', 'about', 'how', 'she', 'would', 'probably', 'find', 'him', 'doing', 'whatever', 'it', 'is', 'you', 'suggested', 'he', 'should', 'have', 'done', 'weird']


 tokenized and lemmatized document: 
(['part', 'couldnt', 'anym', 'irrelev', 'im', 'talk', 'would', 'prob', 'find', 'whatev', 'suggest', 'don', 'weird'], ['part', 'couldnt', 'anymore', 'irrelevant', 'im', 'talk', 'would', 'probably', 'find



 tokenized and lemmatized document: 
(['us', 'stat', 'simil', 'econom', 'develop', 'ign', 'dc', 'gdpscapita', 'rang', '65k', '30k', 'doll', 'europ', 'ign', 'luxemburg', 'gdpscapita', 'rang', '79k', 'one thousand, nine hundred', 'doll', '\n\nits', 'obvy', 'country', 'low', 'gdpscapita', 'ar', 'sensit', 'toward', 'soc', 'issu', 'theyr', 'ev', 'first', 'world', 'country', 'emerg', 'country', 'also', 'cult', 'diff', 'sev', 'cult', 'fact', 'country', '90s', 'totalit', 'govern', 'theref', 'expery', 'democr', 'includ', 'min', 'right', 'ev', 'thirty', 'year', '\n\ntherefore', 'say', 'europ', 'refer', 'country', 'doubl', 'standard', 'lik', 'look', 'singap', 'say', 'as', 'rich'], ['us', 'state', 'similar', 'economically', 'develop', 'ignore', 'dc', 'gdpscapita', 'range', '65k', '30k', 'dollars', 'europe', 'ignore', 'luxemburg', 'gdpscapita', 'range', '79k', 'one thousand, nine hundred', 'dollar', '\n\nits', 'obvious', 'countries', 'low', 'gdpscapita', 'arent', 'sensitive', 'towards', 'social',



 tokenized and lemmatized document: 
(['heard', 'met', 'got', 'hit', 'sound', 'dirty', 'okay', 'guy', 'stil', 'breath', 'pleas', 'tel', 'pric', 'dust', 'block'], ['hear', 'mete', 'get', 'hit', 'sound', 'dirty', 'okay', 'guy', 'still', 'breathe', 'please', 'tell', 'price', 'dust', 'blocker'])
original document: 
['I', 'think', 'being', 'out', 'to', 'each', 'other', 'is', 'really', 'important,', 'so', 'good', 'to', 'hear', 'you', 'both', 'are.\n\nAnd', "I'm", 'certainly', 'not', 'trying', 'to', 'push', 'you', 'into', 'or', 'away', 'from', 'hormones', 'or', 'anything', 'else.', "I'm", 'just', 'warning', 'against', 'slipping', 'into', 'denial', 'or', 'repression.', 'As', 'long', 'as', "you're", 'honest', 'with', 'yourself,', 'you', 'can', 'make', 'an', 'informed', '(if', 'scary!)', 'decision.\n\nFBI...Female', 'But', 'Incognito?', 'I', 'might', 'steal', 'that...']


 tokenized and lemmatized document: 
(['think', 'real', 'import', 'good', 'hear', 'are\n\nand', 'im', 'certain', 'try', 'pu



 tokenized and lemmatized document: 
(['mass', 'park', 'lot', 'pack', 'schlitz', 'park', 'commut', 'monday', 'friday', 'took', 'year', 'get', 'agr', 'less', 'repav', 'big', 'lot', 'imagin', 'jockey', 'requir', 'shut', 'long', 'enough', 'build', 'park', 'gar', 'plu', 'impact', 'pleas', 'cherry', 'construct', 'would', 'incred', 'inconveny', 'resid', 'commut', 'employ', 'alik'], ['massive', 'park', 'lot', 'pack', 'schlitz', 'park', 'commuters', 'monday', 'friday', 'take', 'years', 'get', 'agreement', 'lessees', 'repave', 'big', 'lot', 'imagine', 'jockey', 'require', 'shut', 'long', 'enough', 'build', 'park', 'garage', 'plus', 'impact', 'pleasant', 'cherry', 'construction', 'would', 'incredibly', 'inconvenient', 'residents', 'commute', 'employees', 'alike'])
original document: 
['First,', 'try', 'using', 'your', 'controller', 'in', 'another', 'game', 'and', 'try', 'using', 'another', 'controller,', 'this', 'will', 'determine', 'whether', 'this', 'is', 'a', 'problem', 'with', 'the', 'cont

...



 tokenized and lemmatized document: 
(['iv', 'expery', 'lik', 'passeng', 'upset', 'couldnt', 'drink', 'bud', 'lit', 'car'], ['ive', 'experience', 'like', 'passenger', 'upset', 'couldnt', 'drink', 'bud', 'lite', 'car'])
original document: 
['Red', 'Wings', 'are', 'notorious', 'for', 'a', 'hard', 'break', 'in', 'period.', 'But', 'when', "it's", 'done', 'they', 'create', 'an', 'exact', 'mold', 'to', 'your', 'foot']


 tokenized and lemmatized document: 
(['red', 'wing', 'not', 'hard', 'break', 'period', 'don', 'cre', 'exact', 'mold', 'foot'], ['red', 'wing', 'notorious', 'hard', 'break', 'period', 'do', 'create', 'exact', 'mold', 'foot'])
original document: 
['Depends', 'on', 'who', 'you', 'ask', 'LOL!', 'I', 'like', 'some', 'of', 'it', 'but', 'it', "doesn't", 'taste', 'like', 'the', 'real', 'thing']


 tokenized and lemmatized document: 
(['depend', 'ask', 'lol', 'lik', 'doesnt', 'tast', 'lik', 'real', 'thing'], ['depend', 'ask', 'lol', 'like', 'doesnt', 'taste', 'like', 'real', 'thing



 tokenized and lemmatized document: 
(['sery', 'on', 'cas', 'think', 'op', 'nee', 'ask', 'want', 'maintain', 'fath', 'benefit', 'kid', 'dont', 'know', 'long', 'kid', 'deserv', 'man', 'support', 'fin', 'emot'], ['seriously', 'one', 'case', 'think', 'op', 'need', 'ask', 'want', 'maintain', 'fatherhood', 'benefit', 'kid', 'dont', 'know', 'long', 'kid', 'deserve', 'man', 'support', 'financially', 'emotionally'])
original document: 
['Good', 'bot!']


 tokenized and lemmatized document: 
(['good', 'bot'], ['good', 'bot'])
original document: 
['Cue', 'me', 'going', 'to', 'play', 'fallout', '3.']


 tokenized and lemmatized document: 
(['cue', 'going', 'play', 'fallout', 'three'], ['cue', 'go', 'play', 'fallout', 'three'])
original document: 
['Thank', 'you', 'for', 'bringing', 'this', 'to', 'my', 'attention.', 'I', 'go', 'to', 'this', 'sub', 'every', 'few', 'days', 'and', 'actually', 'care', 'about', 'the', 'alert', 'slider', 'a', 'lot', 'but', 'I', "didn't", 'notice', 'the', 'stickied', '



 tokenized and lemmatized document: 
(['yo', 'assum', 'way', 'much', 'av', 'play', 'hug', 'play', 'bas', 'ther', 'way', 'everyon', 'start', 'gam', 'start', 'peopl', 'stil', 'complain', 'camusxand', 'rep', 'start', 'point', 'ther', 'way', 'everyon', 'goal', 'collect', 'every', 'unit', 'four', 'level', 'forty', 'ther', 'lit', 'point', 'eith', 'thing', 'gam', 'act', 'track', 'fiv', 'level', '40s', 'f2p', 'limit', 'barrack', 'spac', 'limit', 'supply', 'feath', 'ther', 'noth', 'gam', 'ev', 'want', 'iv', 'upgrad', 'lit', 'sev', 'tot', 'unit', 'fiv', 'amount', 'feath', 'iv', 'gath', 'sint', 'start', 'play', 'dont', 'enough', 'upgrad', 'someon', 'fiv', 'right', 'two', 'four', 'pul', 'want', 'build', 'lik', 'ton', 'us', 'would', 'want', 'favorit', 'unit', 'would', 'much', 'import', 'goal', 'rais', 'vir', 'four', 'level', 'forty', 'us', 'feath', 'stamin', 'wouldv', 'otherw', 'put', 'ten', 'selen', 'someth', '\neven', 'somehow', 'sev', 'thousand', 'lik', 'mind', 'individ', 'youd', 'stil', 'abl'



 tokenized and lemmatized document: 
(['httpsmyoutubecomwatchvgrglosbv2ve'], ['httpsmyoutubecomwatchvgrglosbv2ve'])
original document: 
['lets', 'see', 'how', 'deep', 'this', 'rabbit', 'hole', 'goes...']


 tokenized and lemmatized document: 
(['let', 'see', 'deep', 'rabbit', 'hol', 'goe'], ['let', 'see', 'deep', 'rabbit', 'hole', 'go'])
original document: 
['Yes,', 'to', 'buy', 'brand', 'new', 'authentic', 'pairs', 'of', 'both', 'shoes', 'will', 'cost', 'you', 'close', 'to', '2,000', 'USD', 'total.', '']


 tokenized and lemmatized document: 
(['ye', 'buy', 'brand', 'new', 'auth', 'pair', 'sho', 'cost', 'clos', 'two thousand', 'usd', 'tot'], ['yes', 'buy', 'brand', 'new', 'authentic', 'pair', 'shoe', 'cost', 'close', 'two thousand', 'usd', 'total'])
original document: 
['**POTD', 'Record:**', '7-0-1', '(W-P-L)\n\nStreak:', '6W\n\nPick', 'record:', '23-2-16\n\nBonus', 'Picks', 'record:', '17-1-23\n***\n\n*Had', 'a', 'decent', 'day', 'yesterday.', 'The', 'POTD', 'pick', '-', 'Coventry



 tokenized and lemmatized document: 
(['potd', 'record', 'seven hundred and one', 'wpl\n\nstreak', '6w\n\npick', 'record', '23216\n\nbonus', 'pick', 'record', '17123\n\n\nhad', 'dec', 'day', 'yesterday', 'potd', 'pick', 'coventry', 'got', 'forty-six', 'pick', 'win', 'on', 'push\n\npotd\n\ndouble', 'torino', 'win', 'amp', 'int', 'mil', 'win', 'one hundred and sixty', 'l', 'twenty-two', 'amp', 'twelv', 'torino', 'cont', 'two', 'goal', 'last', 'six', 'minut', 'los', 'bet', 'unbelievable\n\nnow', 'lik', 'stick', 'singl', 'espec', 'potd', 'research', 'gam', 'lik', 'today', 'ther', 'noth', 'put', 'edg', 'certain', 'pick', 'think', 'lazio', 'gam', 'o25', 'goal', 'see', 'team', 'scor', 'ev', 'lazio', 'scor', 'three', 'im', 'one hundred', 'confid', 'twenty', 'win', 'gam', 'wouldnt', 'surpr', '\n\nso', 'detail', 'torino', 'gam', 'host', 'veron', 'today', 'iv', 'talk', 'prevy', 'pick', 'ut', 'terr', 'report', 'believ', 'man', 'clos', 'sack', 'veron', 'look', 'hopeless', 'six', 'gam', 'record', 